In [ ]:
# install py-spark
!pip install --quiet pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 58.1 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Start Spark Session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession \
    .builder \
    .appName("BDP-project") \
    .getOrCreate()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# A function to create table from a CSV
def readCSV(file_name):
  df = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep", ",") \
  .csv( "/content/drive/MyDrive/" + file_name )

  return df

In [ ]:
Co2_file=readCSV("CO2 Emissions_Canada.csv")
fuel_file=readCSV("Fuel_data.csv")

In [ ]:
# A utility function used for printing a dataset
def print_df(df):
  # dump content of dataset one on a line
  xlist = df.toPandas()
  print( xlist )

In [ ]:
Co2_file.createOrReplaceTempView( "Co2" )
fuel_file.createOrReplaceTempView("Fuel")

#Origin Of the 3D Cube

In [ ]:
Co2_file.agg(sum("CO2_Emissions").alias("Total CO2 produced")).show()

+------------------+
|Total CO2 produced|
+------------------+
|           1850568|
+------------------+



#1D Cube For **"Make"** Dimension

In [ ]:
Co2_file.createOrReplaceTempView( "df" )

In [ ]:
Co2_file.cube("Make").agg(sum("CO2_Emissions").alias("Total CO2 produced by Make")).orderBy("Make").show()

# q1="Select Make,sum(CO2_Emissions) AS Total_CO2_produced_by_Make from df "\
#     + "GROUP BY Make WITH CUBE "\
#     + "ORDER BY Make"
# spark.sql(q1).show()

+------------+--------------------------+
|        Make|Total CO2 produced by Make|
+------------+--------------------------+
|        null|                   1850568|
|       ACURA|                     16103|
|  ALFA ROMEO|                      6747|
|ASTON MARTIN|                     15962|
|        AUDI|                     71742|
|     BENTLEY|                     16695|
|         BMW|                    133862|
|     BUGATTI|                      1566|
|       BUICK|                     24005|
|    CADILLAC|                     42005|
|   CHEVROLET|                    155436|
|    CHRYSLER|                     21663|
|       DODGE|                     67985|
|        FIAT|                     14034|
|        FORD|                    163901|
|     GENESIS|                      7121|
|         GMC|                     98373|
|       HONDA|                     40707|
|     HYUNDAI|                     43863|
|    INFINITI|                     27439|
+------------+--------------------

#2D Cube for **"Make,Model"** Dimensions

In [ ]:
Co2_file.cube("Make","Model").agg(sum("CO2_Emissions").alias("Total CO2 produced by Make")).orderBy("Make","Model").show()

# q2="Select Make,Model,sum(CO2_Emissions) AS Total_CO2_produced_by_Make from df "\
#     + "GROUP BY Make,Model WITH CUBE "\
#     + "ORDER BY Make,Model"
# spark.sql(q2).show()

+----+--------------------+--------------------------+
|Make|               Model|Total CO2 produced by Make|
+----+--------------------+--------------------------+
|null|                null|                   1850568|
|null|          124 SPIDER|                       744|
|null|          124 Spider|                       744|
|null|                1500|                      2244|
|null|          1500 (MDS)|                      1973|
|null|            1500 4X4|                      2339|
|null|      1500 4X4 (MDS)|                      2048|
|null|     1500 4X4 DIESEL|                       286|
|null|  1500 4X4 ECODIESEL|                      1147|
|null|  1500 4X4 EcoDiesel|                       260|
|null|        1500 4X4 FFV|                      2928|
|null|    1500 4X4 eTorque|                      1121|
|null|        1500 Classic|                      1264|
|null|    1500 Classic 4X4|                      1334|
|null|1500 Classic 4X4 ...|                       289|
|null|1500

In [ ]:
Co2_fuel=spark.sql("Select * from Co2 c join Fuel f on (c.ID==f.Fuel_ID)")

#1.List Make which vehicle class is SUV-Small and fuel consuption city is more than 12(L/100km).

In [ ]:
Co2_fuel.createOrReplaceTempView( "Co2_fuel" )

In [ ]:
#2D Dicing 
spark.sql("Select Make,Vehicle_Class,Fuel_ConsumptionCity,CO2_Emissions from Co2_fuel where Vehicle_Class='SUV - SMALL' and Fuel_ConsumptionCity>12 group by Make,Vehicle_Class,Fuel_ConsumptionCity,CO2_Emissions").show()

+-------------+-------------+--------------------+-------------+
|         Make|Vehicle_Class|Fuel_ConsumptionCity|CO2_Emissions|
+-------------+-------------+--------------------+-------------+
|   LAND ROVER|  SUV - SMALL|                15.2|          318|
|   VOLKSWAGEN|  SUV - SMALL|                13.7|          282|
|       NISSAN|  SUV - SMALL|                16.0|          327|
|MERCEDES-BENZ|  SUV - SMALL|                17.4|          353|
|         FORD|  SUV - SMALL|                13.7|          281|
|    CHEVROLET|  SUV - SMALL|                16.3|          234|
|     INFINITI|  SUV - SMALL|                12.5|          257|
|      LINCOLN|  SUV - SMALL|                12.6|          259|
|    CHEVROLET|  SUV - SMALL|                14.9|          300|
|        HONDA|  SUV - SMALL|                13.0|          265|
|        ACURA|  SUV - SMALL|                12.2|          258|
|          KIA|  SUV - SMALL|                12.5|          264|
|        DODGE|  SUV - SM

#2.List sum of Co2_emission for all audi vehicles which fuel consuption is less than 20 and Vehicle_Class has to be Two-Seater.


In [ ]:
#Drill Down 3D
spark.sql("Select Make,Model,Vehicle_Class,sum(CO2_Emissions) as Co2_sum from Co2_fuel where Fuel_ConsumptionCity<20 and Make='AUDI' and Vehicle_Class='TWO-SEATER' group by Make,Model,Vehicle_Class").show(36)

+----+--------------------+-------------+-------+
|Make|               Model|Vehicle_Class|Co2_sum|
+----+--------------------+-------------+-------+
|AUDI|TTS ROADSTER QUATTRO|   TWO-SEATER|    472|
|AUDI|           R8 Spyder|   TWO-SEATER|    347|
|AUDI|          R8 QUATTRO|   TWO-SEATER|    338|
|AUDI|                  R8|   TWO-SEATER|   2368|
|AUDI| TT ROADSTER QUATTRO|   TWO-SEATER|   1048|
|AUDI| TT Roadster quattro|   TWO-SEATER|    426|
|AUDI|            R8 Coupe|   TWO-SEATER|    347|
|AUDI|           R8 SPYDER|   TWO-SEATER|   2402|
+----+--------------------+-------------+-------+



#3.List sum of CO2_Emissions for all the models of CHEVROLET which engine size is 6.2 ltr and fuel concomb(mpg) is grater than 22.

In [ ]:
#Dicing with CUBE
q3="Select Make,Model,sum(CO2_Emissions) as Co2_sum from Co2_fuel "\
  +"where Make=='CHEVROLET' and Engine_Size=6.2 and Fuel_ConsumptionComb_mpg>22 " \
  +"group by Make,Model with CUBE order by Make,Co2_sum asc"
spark.sql(q3).show()

+---------+---------+-------+
|     Make|    Model|Co2_sum|
+---------+---------+-------+
|     null|Camaro SS|    561|
|     null| Corvette|    576|
|     null|CAMARO SS|   1670|
|     null| CORVETTE|   2445|
|     null|     null|   5252|
|CHEVROLET|Camaro SS|    561|
|CHEVROLET| Corvette|    576|
|CHEVROLET|CAMARO SS|   1670|
|CHEVROLET| CORVETTE|   2445|
|CHEVROLET|     null|   5252|
+---------+---------+-------+



#4.List the Make,vehicle class,sum of fuel Consumption on highway of mercedes-benz

In [ ]:
#Slicing With Roll Up
spark.sql("Select Make,Vehicle_Class,round(sum(Fuel_ConsumptionHwy),4) as sum_fuelhwy from Co2_fuel where Make='MERCEDES-BENZ' group by Make,Vehicle_Class with rollup order by sum_fuelhwy desc").show()

+-------------+--------------------+-----------+
|         Make|       Vehicle_Class|sum_fuelhwy|
+-------------+--------------------+-----------+
|MERCEDES-BENZ|                null|     4103.2|
|         null|                null|     4103.2|
|MERCEDES-BENZ|      SUV - STANDARD|      909.5|
|MERCEDES-BENZ|             COMPACT|      693.9|
|MERCEDES-BENZ|          SUBCOMPACT|      636.1|
|MERCEDES-BENZ|          TWO-SEATER|      556.4|
|MERCEDES-BENZ|            MID-SIZE|      393.4|
|MERCEDES-BENZ|           FULL-SIZE|      315.8|
|MERCEDES-BENZ|         SUV - SMALL|      235.0|
|MERCEDES-BENZ|STATION WAGON - M...|      171.1|
|MERCEDES-BENZ|SPECIAL PURPOSE V...|       87.9|
|MERCEDES-BENZ|STATION WAGON - S...|       82.3|
|MERCEDES-BENZ|         MINICOMPACT|       21.8|
+-------------+--------------------+-----------+



In [ ]:
Co2_fuel.show()

+---+------------+------------+-------------+-----------+---------+------------+-------------+-------+---------+--------------------+-------------------+--------------------+------------------------+
| ID|        Make|       Model|Vehicle_Class|Engine_Size|Cylinders|Transmission|CO2_Emissions|Fuel_ID|Fuel_Type|Fuel_ConsumptionCity|Fuel_ConsumptionHwy|Fuel_ConsumptionComb|Fuel_ConsumptionComb_mpg|
+---+------------+------------+-------------+-----------+---------+------------+-------------+-------+---------+--------------------+-------------------+--------------------+------------------------+
|  1|       ACURA|         ILX|      COMPACT|        2.0|        4|         AS5|          196|      1|        Z|                 9.9|                6.7|                 8.5|                      33|
|  2|       ACURA|         ILX|      COMPACT|        2.4|        4|          M6|          221|      2|        Z|                11.2|                7.7|                 9.6|                      29|


#5.List all make and model of vehicle which have minimum 4L engine size and atleast 4 cylinders engine.

In [ ]:
#Dicing 2D with Roll Up
q5="Select Make,Model,sum(CO2_Emissions) as Co2_sum from Co2_fuel " \
  +"where Engine_Size>4 and Cylinders>=4 "\
  +"group by Make,Model WITH ROLLUP order by Make,Model,Co2_sum"
spark.sql(q5).show()

+------------+----------------+-------+
|        Make|           Model|Co2_sum|
+------------+----------------+-------+
|        null|            null| 492616|
|ASTON MARTIN|            null|  14282|
|ASTON MARTIN|        DB11 AMR|    624|
|ASTON MARTIN|        DB11 V12|    644|
|ASTON MARTIN|             DB9|    718|
|ASTON MARTIN|          DB9 GT|    365|
|ASTON MARTIN|DBS Superleggera|    648|
|ASTON MARTIN|          RAPIDE|    359|
|ASTON MARTIN|        RAPIDE S|   1322|
|ASTON MARTIN|      Rapide AMR|    332|
|ASTON MARTIN|   V12 VANTAGE S|   1607|
|ASTON MARTIN|      V8 VANTAGE|   2091|
|ASTON MARTIN|    V8 VANTAGE S|   2091|
|ASTON MARTIN|        VANQUISH|   1736|
|ASTON MARTIN|      VANTAGE GT|   1399|
|ASTON MARTIN| Vanquish Zagato|    346|
|        AUDI|            null|  10454|
|        AUDI|             A8L|    707|
|        AUDI|     A8L QUATTRO|    344|
|        AUDI|              R8|   3566|
+------------+----------------+-------+
only showing top 20 rows



#6.List model and vehicle class,total of Fuel_ConsumptionCity of Porsche which fuel consumption comb is more than 10(L/100) and CO2_Emissions more than 251.

In [ ]:
#2D Dicing
q6="Select Make,Model,round(sum(Fuel_ConsumptionCity),2) as fuel_sum from Co2_fuel "\
  +"where Make='PORSCHE' and Fuel_ConsumptionComb>10 and CO2_Emissions>251 "\
  +"group by Make,Model With ROLLUP order by fuel_sum desc"
spark.sql(q6).show()

+-------+--------------------+--------+
|   Make|               Model|fuel_sum|
+-------+--------------------+--------+
|   null|                null|  2641.9|
|PORSCHE|                null|  2641.9|
|PORSCHE|             911 GT3|   115.6|
|PORSCHE|       CAYENNE TURBO|    83.1|
|PORSCHE|911 CARRERA 4S CA...|    76.4|
|PORSCHE|           CAYENNE S|    71.9|
|PORSCHE|             CAYENNE|    69.2|
|PORSCHE|     CAYENNE TURBO S|    68.3|
|PORSCHE| 911 TURBO CABRIOLET|    66.3|
|PORSCHE|         911 TURBO S|    66.3|
|PORSCHE|911 TURBO S CABRI...|    66.3|
|PORSCHE|           911 TURBO|    66.3|
|PORSCHE|911 CARRERA 4 GTS...|    65.9|
|PORSCHE|911 CARRERA GTS C...|    64.8|
|PORSCHE|     911 CARRERA GTS|    64.1|
|PORSCHE|   911 CARRERA 4 GTS|    64.1|
|PORSCHE|         CAYENNE GTS|    60.5|
|PORSCHE|PANAMERA TURBO EX...|    59.0|
|PORSCHE|      PANAMERA TURBO|    58.7|
|PORSCHE|         MACAN TURBO|    56.8|
+-------+--------------------+--------+
only showing top 20 rows



#7.List make,vehicle class where transmission is AS6 and average co2(g/km).

In [ ]:
#Slice
q7="Select Make,Vehicle_Class,round(avg(CO2_Emissions),2) as CO2_avg from Co2_fuel"\
   +" where Transmission='AS6' "\
   +"group by Make,Vehicle_Class order by CO2_avg desc"
spark.sql(q7).show()

+----------+--------------------+-------+
|      Make|       Vehicle_Class|CO2_avg|
+----------+--------------------+-------+
|    TOYOTA|PICKUP TRUCK - ST...| 359.38|
|  MASERATI|          SUBCOMPACT| 353.91|
|    TOYOTA|      SUV - STANDARD|  348.4|
|      FORD|     VAN - PASSENGER| 344.38|
|     LEXUS|      SUV - STANDARD| 342.67|
|   LINCOLN|      SUV - STANDARD| 323.17|
|    JAGUAR|          TWO-SEATER|  311.0|
|    JAGUAR|         MINICOMPACT|  308.3|
|     VOLVO|      SUV - STANDARD|  294.0|
|      FORD|PICKUP TRUCK - ST...| 292.21|
|      FORD|      SUV - STANDARD| 290.86|
|  CADILLAC|      SUV - STANDARD| 290.17|
|  CADILLAC|STATION WAGON - S...|  282.4|
|   LINCOLN|           FULL-SIZE| 281.67|
|       KIA|             MINIVAN| 278.08|
|  CADILLAC|           FULL-SIZE| 276.44|
| CHEVROLET|          TWO-SEATER|  276.0|
|    TOYOTA|             MINIVAN| 275.33|
|  CADILLAC|            MID-SIZE|  274.9|
|LAND ROVER|         SUV - SMALL|  274.0|
+----------+--------------------+-

#8.List Make, Model and Vehicle class where fuel type is X and fuel consumption comb is min 45(mpg) .

In [ ]:
spark.sql("Select Make,Model,Vehicle_Class from Co2_fuel where Fuel_Type='X' and Fuel_ConsumptionComb_mpg>45").show()

+----------+-------------------+--------------------+
|      Make|              Model|       Vehicle_Class|
+----------+-------------------+--------------------+
|      FORD|       C-MAX HYBRID|           FULL-SIZE|
|      FORD|      FUSION HYBRID|            MID-SIZE|
|     HONDA|      ACCORD HYBRID|            MID-SIZE|
|   HYUNDAI|      SONATA HYBRID|            MID-SIZE|
|       KIA|      OPTIMA HYBRID|            MID-SIZE|
|     LEXUS|            CT 200h|             COMPACT|
|     LEXUS|            ES 300h|            MID-SIZE|
|MITSUBISHI|             MIRAGE|             COMPACT|
|     SCION|                 iQ|         MINICOMPACT|
|    TOYOTA|    CAMRY HYBRID LE|            MID-SIZE|
|    TOYOTA|CAMRY HYBRID XLE/SE|            MID-SIZE|
|    TOYOTA|              PRIUS|            MID-SIZE|
|    TOYOTA|            PRIUS c|             COMPACT|
|    TOYOTA|            PRIUS v|STATION WAGON - M...|
|      FORD|       C-MAX HYBRID|           FULL-SIZE|
|      FORD|      FUSION HYB

#9.List Fuel Consumption in City With Make,Model Dimensions 

In [ ]:
#Drill Down 2D
spark.sql("Select Make,Model,Fuel_ConsumptionCity from Co2_fuel group by Make,Model,Fuel_ConsumptionCity With rollup order by Fuel_ConsumptionCity desc").show()

+-----------+--------------------+--------------------+
|       Make|               Model|Fuel_ConsumptionCity|
+-----------+--------------------+--------------------+
|  CHEVROLET|EXPRESS 3500 PASS...|                30.6|
|        GMC|SAVANA 3500 PASSE...|                30.6|
|        GMC|SAVANA 2500 PASSE...|                30.3|
|  CHEVROLET|EXPRESS 2500 PASS...|                30.3|
|  CHEVROLET|EXPRESS 3500 PASS...|                30.2|
|        GMC|SAVANA 3500 PASSE...|                30.2|
|  CHEVROLET|EXPRESS 2500 PASS...|                30.0|
|        GMC|SAVANA 2500 PASSE...|                30.0|
|    BUGATTI|              Chiron|                26.8|
|    BUGATTI|              CHIRON|                26.8|
|       FORD|      E350 WAGON FFV|                26.7|
|LAMBORGHINI|  Aventador Roadster|                26.6|
|LAMBORGHINI|     Aventador Coupe|                26.3|
|LAMBORGHINI|     Aventador Coupe|                26.2|
|   CADILLAC|    ESCALADE ESV AWD|              

In [ ]:
Co2_fuel.show()

+---+------------+------------+-------------+-----------+---------+------------+-------------+-------+---------+--------------------+-------------------+--------------------+------------------------+
| ID|        Make|       Model|Vehicle_Class|Engine_Size|Cylinders|Transmission|CO2_Emissions|Fuel_ID|Fuel_Type|Fuel_ConsumptionCity|Fuel_ConsumptionHwy|Fuel_ConsumptionComb|Fuel_ConsumptionComb_mpg|
+---+------------+------------+-------------+-----------+---------+------------+-------------+-------+---------+--------------------+-------------------+--------------------+------------------------+
|  1|       ACURA|         ILX|      COMPACT|        2.0|        4|         AS5|          196|      1|        Z|                 9.9|                6.7|                 8.5|                      33|
|  2|       ACURA|         ILX|      COMPACT|        2.4|        4|          M6|          221|      2|        Z|                11.2|                7.7|                 9.6|                      29|


#10.List 3D Drill Down with sum of Fuel Consumption of vehicle Combination of city and highway(L/100km)

In [ ]:
spark.sql("Select Make,Model,Vehicle_Class,round(sum(Fuel_ConsumptionComb),5) as fuel_sum from Co2_fuel group by Make,Model,Vehicle_Class with rollup order by fuel_sum desc").show()

+-------------+-----+-------------+--------+
|         Make|Model|Vehicle_Class|fuel_sum|
+-------------+-----+-------------+--------+
|         null| null|         null| 81050.9|
|         FORD| null|         null|  7537.6|
|    CHEVROLET| null|         null|  7040.7|
|          BMW| null|         null|  5725.5|
|MERCEDES-BENZ| null|         null|  4993.3|
|          GMC| null|         null|  4577.5|
|      PORSCHE| null|         null|  4176.1|
|       TOYOTA| null|         null|  3181.4|
|        DODGE| null|         null|  3077.5|
|         AUDI| null|         null|  3060.6|
|         JEEP| null|         null|  2736.2|
|       NISSAN| null|         null|  2627.2|
|          KIA| null|         null|  2116.5|
|      HYUNDAI| null|         null|  1874.7|
|       JAGUAR| null|         null|  1864.2|
|        LEXUS| null|         null|  1844.0|
|     CADILLAC| null|         null|  1815.9|
|   VOLKSWAGEN| null|         null|  1750.2|
|        HONDA| null|         null|  1748.2|
|         